In [490]:
import json
from collections import Counter
from tqdm.notebook import tqdm
import random
from random import sample, seed, shuffle

import re
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/hjkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hjkim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hjkim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [51]:
# 예시 데이터셋
with open('dataset.json', 'r') as fp:
    data = json.load(fp)
len(data)

21855

In [538]:
with open('dataset/amazon_laptop.json', 'r') as fp:
    data = json.load(fp)
len(data)

271846

In [543]:
with open('dataset/yelp_rest.json', 'r') as fp:
    data = json.load(fp)
len(data)

270000

In [540]:
data[9]

'I just feel the Apple is OK. I have not used it. I got it all wrong that it has a DVD drive, but no it is s CD Drive. The specs did not show clearly that it has no DVD drive. I hope I can learn more about apple using this.'

In [4]:
domains = [i['domain'] for i in data]
set(domains)

{'apparel',
 'automotive',
 'baby',
 'beauty',
 'books',
 'camera_&_photo',
 'cell_phones_&_service',
 'computer_&_video_games',
 'dvd',
 'electronics',
 'grocery',
 'health_&_personal_care',
 'jewelry_&_watches',
 'kitchen_&_housewares',
 'magazines',
 'music',
 'office_products',
 'outdoor_living',
 'software',
 'sports_&_outdoors',
 'toys_&_games',
 'video'}

In [111]:
# electronics domain만 추출
data = [review['text'] for review in data if review['domain']=='electronics']
len(data)

1277

In [544]:
# sentence tokenization
data_sent = []

for review in data:
    data_sent.extend(sent_tokenize(str(review))) # str 안 넣으면 에러 뜸
len(data_sent)

2102715

In [542]:
with open('amazon_laptop_sent.json', 'w') as fp:
    json.dump(data_sent, fp)

In [535]:
with open('yelp_rest_sent.json', 'w') as fp:
    json.dump(data_sent, fp)

In [545]:
data_sent[:10]

['Pretty good BBQ and decent service.',
 'The side dishes are just "OK".',
 'Nothing special about this place.',
 'Nothing really stands out.',
 'You can get a three meat combo with two sides for around $18.',
 'With ribs you have to add $1.',
 "I'd go back for lunch but probably not for dinner.",
 'Their wine selection is horrible.',
 'Better to eat with a beer.',
 "A little noisy but not so bad you can't have a conversation."]

In [483]:
word_tokenize(data_sent[0])

['To',
 'put',
 'it',
 'simply',
 ',',
 'Apple',
 "'s",
 'new',
 'iBook',
 'is',
 'an',
 'engineering',
 'marvel',
 '.']

In [146]:
# sentence tokenization (with POS)

# data_sent_pos = [pos_tag(word_tokenize(review)) for review in data_sent]
# len(data_sent_pos)

4642

In [123]:
# data_sent_pos[:10]

[[('I', 'PRP'),
  ('received', 'VBD'),
  ('my', 'PRP$'),
  ('Kingston', 'NNP'),
  ('256MB', 'CD'),
  ('SD', 'NNP'),
  ('card', 'NN'),
  ('just', 'RB'),
  ('as', 'IN'),
  ('advertised.The', 'JJ'),
  ('unit', 'NN'),
  ('came', 'VBD'),
  ('in', 'IN'),
  ('the', 'DT'),
  ('mail', 'NN'),
  ('exactly', 'RB'),
  ('2', 'CD'),
  ('days', 'NNS'),
  ('after', 'IN'),
  ('Iordered', 'NNP'),
  ('.', '.')],
 [('Worked', 'VBN'),
  ('perfectly', 'RB'),
  ('and', 'CC'),
  ('I', 'PRP'),
  ("'m", 'VBP'),
  ('very', 'RB'),
  ('satisfied', 'JJ')],
 [('Works', 'NNP'),
  ('well', 'RB'),
  (',', ','),
  ('especially', 'RB'),
  ('for', 'IN'),
  ('anyone', 'NN'),
  ('who', 'WP'),
  ('still', 'RB'),
  ('has', 'VBZ'),
  ('their', 'PRP$'),
  ('old', 'JJ'),
  ('console', 'JJ'),
  ('systems', 'NNS'),
  ('in', 'IN'),
  ('use', 'NN'),
  ('.', '.')],
 [('I', 'PRP'),
  ('have', 'VBP'),
  ('my', 'PRP$'),
  ('NES', 'NNP'),
  ('and', 'CC'),
  ('SNES', 'NNP'),
  (',', ','),
  ('both', 'DT'),
  ('of', 'IN'),
  ('which', 'WDT'

In [113]:
# sentence tokenization 제대로 안 된 경우
# 마침표 뒤에 띄어쓰기가 빠진 경우

print(data[0])
print()
for i in sent_tokenize(data[0]):
    print(i)
print()
print(data[1])
print()
for i in sent_tokenize(data[1]):
    print(i)

I received my Kingston 256MB SD card just as advertised.The unit came in the mail exactly 2 days after Iordered. Worked perfectly and I'm very satisfied

I received my Kingston 256MB SD card just as advertised.The unit came in the mail exactly 2 days after Iordered.
Worked perfectly and I'm very satisfied

Works well, especially for anyone who still has their old console systems in use. I have my NES and SNES, both of which have huge AC adapters, plugged into this. It saves tons of space on my power strip.The design is a little clumsy though, so it's only getting 4/5

Works well, especially for anyone who still has their old console systems in use.
I have my NES and SNES, both of which have huge AC adapters, plugged into this.
It saves tons of space on my power strip.The design is a little clumsy though, so it's only getting 4/5


In [ ]:
# word tokenization
# data_word = []

# for sent in data_sent:
#     data_word.extend(word_tokenize(sent))
# len(data_word)

In [124]:
# Counter(data_word).most_common()

[('.', 3279),
 ('the', 2641),
 ('I', 2168),
 (',', 1961),
 ('and', 1708),
 ('to', 1600),
 ('a', 1482),
 ('it', 1385),
 ('is', 1163),
 ('for', 876),
 ('this', 810),
 ('of', 774),
 ('my', 704),
 ('with', 639),
 ('in', 588),
 ('that', 536),
 ('have', 520),
 ('not', 511),
 ('was', 486),
 ("n't", 477),
 ('on', 475),
 ('you', 460),
 ('The', 452),
 ('but', 430),
 ('!', 411),
 ('It', 313),
 ('This', 296),
 ("'s", 280),
 ('as', 266),
 ('very', 265),
 ('product', 263),
 ('one', 262),
 ('are', 255),
 ('great', 247),
 ('be', 241),
 ('use', 238),
 ('would', 229),
 ('at', 220),
 ('had', 219),
 ('from', 214),
 ('do', 211),
 ('good', 207),
 ('only', 204),
 ('(', 203),
 (')', 202),
 ('has', 200),
 ('just', 198),
 ('they', 194),
 ('can', 192),
 ('all', 192),
 ('get', 185),
 ('an', 182),
 ('so', 179),
 ('your', 179),
 ('bought', 176),
 ('up', 176),
 ('sound', 175),
 ('these', 175),
 ('out', 174),
 ('work', 172),
 ('no', 169),
 ('or', 168),
 ('me', 157),
 ('about', 155),
 ('will', 155),
 ('price', 153),
 

In [546]:
# word tokenization 
# 명사, 동사, 형용사만
# 불용어 제거
# 특수문자가 포함된 단어는 제거

data_keyword = []
pos_list = ['NN', 'NNS', 'NNP', 'NNPS', # 명사
               'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', # 동사
               'JJ', 'JJR', 'JJS'] # 형용사
stopwords_eng = stopwords.words('english')

for sent in tqdm(data_sent):
    for vocab in pos_tag(word_tokenize(sent)):
        # if vocab[1] in pos_list and vocab[0].lower() not in stopwords_eng and not vocab[0].startswith("'"): 
        if vocab[1] in pos_list and vocab[0].lower() not in stopwords_eng and re.search('[^A-Za-z0-9]+', vocab[0]) == None:
            data_keyword.append(vocab[0])
print(len(data_keyword))
print(len(set(data_keyword)))

  0%|          | 0/2102715 [00:00<?, ?it/s]

12077378
142646


In [523]:
with open('amazon_laptop_vocab.json', 'w') as fp:
    json.dump(Counter(data_keyword).most_common(), fp)

In [548]:
with open('yelp_rest_vocab.json', 'w') as fp:
    json.dump(Counter(data_keyword).most_common(), fp)

In [547]:
Counter(data_keyword).most_common()

[('food', 128440),
 ('place', 120004),
 ('good', 118488),
 ('great', 94919),
 ('time', 85912),
 ('get', 79931),
 ('service', 77256),
 ('go', 63744),
 ('got', 51137),
 ('order', 42838),
 ('nice', 42145),
 ('ordered', 41919),
 ('staff', 40499),
 ('best', 38236),
 ('restaurant', 37847),
 ('came', 36658),
 ('delicious', 36007),
 ('try', 34305),
 ('people', 34123),
 ('experience', 34050),
 ('little', 33624),
 ('menu', 33139),
 ('chicken', 32625),
 ('went', 32471),
 ('made', 32458),
 ('come', 32127),
 ('love', 30882),
 ('make', 30394),
 ('said', 30323),
 ('recommend', 28879),
 ('amazing', 28617),
 ('going', 28590),
 ('know', 28300),
 ('wait', 28212),
 ('day', 27801),
 ('friendly', 27589),
 ('way', 27259),
 ('bar', 27151),
 ('first', 26847),
 ('night', 25270),
 ('new', 25232),
 ('want', 24911),
 ('area', 24786),
 ('told', 24780),
 ('take', 24522),
 ('minutes', 24452),
 ('say', 24201),
 ('cheese', 24093),
 ('fresh', 23611),
 ('sure', 23325),
 ('Great', 22918),
 ('table', 22828),
 ('pizza', 225

In [458]:
Counter(data_keyword).most_common()

[('product', 263),
 ('great', 247),
 ('use', 238),
 ('good', 207),
 ('get', 185),
 ('bought', 176),
 ('sound', 175),
 ('work', 172),
 ('price', 153),
 ('quality', 149),
 ('works', 138),
 ('buy', 133),
 ('time', 128),
 ('card', 123),
 ('player', 109),
 ('used', 94),
 ('printer', 89),
 ('money', 88),
 ('case', 86),
 ('problem', 86),
 ('item', 85),
 ('got', 85),
 ('computer', 83),
 ('purchased', 82),
 ('unit', 81),
 ('cable', 81),
 ('worked', 80),
 ('thing', 80),
 ('Amazon', 79),
 ('easy', 77),
 ('better', 76),
 ('phone', 75),
 ('little', 70),
 ('new', 70),
 ('recommend', 69),
 ('using', 68),
 ('need', 66),
 ('fine', 65),
 ('battery', 64),
 ('CD', 63),
 ('TV', 62),
 ('DVD', 62),
 ('batteries', 60),
 ('set', 58),
 ('way', 57),
 ('small', 57),
 ('months', 57),
 ('laptop', 56),
 ('speakers', 55),
 ('system', 54),
 ('go', 54),
 ('made', 54),
 ('best', 54),
 ('problems', 53),
 ('nice', 53),
 ('software', 52),
 ('able', 52),
 ('first', 51),
 ('love', 51),
 ('find', 51),
 ('want', 51),
 ('power'